In [1]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from tagger.core.mongo.models.order import Order
from pytz import timezone
from datetime import datetime
from tagger.models.inventory import Inventory, InventoryStatus


TZ = timezone("Asia/Seoul")
# TARGETS = [
#     "61812c7d26685ff051754cee",
# ]

PENDING_FOR_BUNDLING = [
]

to_be_shipped = Inventory.objects.filter(
    status=InventoryStatus.SHIPPING_PENDING
).exclude(
    out_order_id__in=PENDING_FOR_BUNDLING
).all()

TARGETS = [x.out_order_id for x in to_be_shipped]


CJ_COLUMNS = [
    "주문번호", "주문일", "수령자명", "수령자 우편번호", "수령자 연락처", "수령자주소", "상품명", "옵션명", "수량", "배송메모", "박스타입"
]



def get_cj_row(order):
    payment = order.payment
    return [
        order.code.replace("ORD-", ""),
        order.created.astimezone(TZ).date().isoformat().replace("-", "/"),
        payment.buyername,
        payment.buyerpostcode,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        payment.buyeraddress,
        payment.name,
        ",".join([f"{item.size}/{item.quantity}EA" for item in order.orders]),
        1,
        order.memo,
        "소"
    ]

def get_cu_row(order):
    payment = order.get_payment()
    return [
        payment.buyername,
        payment.buyerpostcode,
        payment.buyeraddress,
        payment.buyeraddress,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        "",
        order.memo,
        "선불"
    ]

def copy_cu():
    orders = Order.objects(id__in=TARGETS)
    df = pd.DataFrame([get_cu_row(order) for order in orders])
    df[1] = df[1].apply(lambda x: x.zfill(5))
    df.to_clipboard()

def make_cj(orders):
    raw_items = [get_cj_row(order) for order in orders]
    
    # 합배처리
    item_map = {}
    for item in raw_items:
        key = (item[2], item[3], item[4])  # name, post, mobile
        if key not in item_map:
            item_map[key] = item
        else:
            new_item = [*item_map[key]]
            new_item[0] = new_item[0] + item[0]  # order code
            new_item[6] = new_item[6] + ", " + item[6]  # payment name
            new_item[7] = new_item[7] + ", " + item[7]  # sizes
            item_map[key] = new_item
            
        
    df = pd.DataFrame(item_map.values(), columns=CJ_COLUMNS).sort_values(by="수령자명")
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open(f"../Downloads/cj_upload_{datetime.now().date().isoformat()}.xlsx", "wb") as file:
        df.to_excel(file, index=None)


In [2]:
shipment_redy = Order.objects(id__in=TARGETS).all()
for o in shipment_redy:
    print(o.payment.buyername, f"https://office.alloff.co/orders/{o.id}")
len(shipment_redy)

황태실 https://office.alloff.co/orders/617bcaf817fc2e5a1f9eade4
한미나 https://office.alloff.co/orders/617bed3017fc2e5a1f9eae91
조정수 https://office.alloff.co/orders/6187f34826685ff051755a02
김효영 https://office.alloff.co/orders/618882f226685ff051755a6c
김은지 https://office.alloff.co/orders/618d019d26685ff051756218
박은숙  https://office.alloff.co/orders/618dce1826685ff05175634a


6

In [3]:
make_cj(shipment_redy)

'43D3W'

In [2]:
Inventory.objects.filter(out_order_id="618ccea526685ff0517561b5").first()

<Inventory: Inventory #167 [SHIPPED] 퀄팅 구스다운 점퍼  (211112-023-R)>